In [21]:
from google.colab import files
uploaded = files.upload()

Saving Base de Datos II Estudio Nacional de la Discapacidad (version 1).csv to Base de Datos II Estudio Nacional de la Discapacidad (version 1) (1).csv
Saving 230630_Base de datos ENDIDE 2022_adultos cuidadores y NNA.csv to 230630_Base de datos ENDIDE 2022_adultos cuidadores y NNA (1).csv


In [22]:
import pandas as pd

# 1. Cargar de ambos CSV
df_endide = pd.read_csv(
    '230630_Base de datos ENDIDE 2022_adultos cuidadores y NNA.csv',
    sep=';',
    encoding='ISO-8859-1'
)
df_endis = pd.read_csv(
    'Base de Datos II Estudio Nacional de la Discapacidad (version 1).csv',
    sep=';',
    encoding='ISO-8859-1'
)

# 2. Normalizar nombres de columnas a snake_case
for temp_df in (df_endide, df_endis):
    temp_df.columns = (
        temp_df.columns
               .str.strip()
               .str.lower()
               .str.replace(r'[ \-]+', '_', regex=True)
    )

# 3. Combinar usando columnas comunes
common_cols = list(set(df_endide.columns) & set(df_endis.columns))
df = pd.concat([df_endide[common_cols], df_endis[common_cols]], ignore_index=True)

# 4. Imputación
# 4.1 Numéricos → mediana
num_cols = df.select_dtypes(include='number').columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
# 4.2 Categóricos → 'Desconocido'
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('Desconocido')

# 5. Crear variable de grupo de edad (si existe)
if 'edad' in df.columns:
    df['grupo_edad'] = pd.cut(
        df['edad'],
        bins=[0, 12, 18, 65, 120],
        labels=['Niño/a', 'Adolescente', 'Adulto/a', 'Adulto mayor']
    )

# 6.1 DataFrame con conteo por región y sexo
df_region_sexo = (
    df
    .groupby(['region', 'sexo'])
    .size()
    .reset_index(name='n_personas')
)

# 6.2 DataFrame con pivot de discapacidad por región
df_disc_pivot = df.pivot_table(
    index='region',
    columns='disc_adulto',
    aggfunc='size',
    fill_value=0
)

from IPython.display import display
display(df_region_sexo)
display(df_disc_pivot)

# 7. Guardar resultado en CSV
df.to_csv('dataset_preparado_simple.csv', index=False)


,region,sexo,n_personas
0,1,1,1440
1,1,2,1593
2,2,1,2072
3,2,2,2439
4,3,1,1636
5,3,2,1946
6,4,1,2101
7,4,2,2539
8,5,1,4836
9,5,2,5701


disc_adulto,#¡NULO!,0,1
region,,,
1,1886,1001,146
2,2566,1759,186
3,2224,1063,295
4,2581,1756,303
5,6014,3645,878
6,3772,1914,443
7,3752,2272,470
8,6663,3642,1149
9,3687,2051,603


# Informe
1. Descripción general del proceso aplicado.

2. Problemas detectados y cómo se abordaron.

3. Preparación final del dataset y posibles usos posteriores.

**1.Descripción general del proceso aplicado**

El análisis comenzó cargando dos fuentes de datos en formato CSV, correspondientes a la encuesta ENDIDE 2022 sobre adultos cuidadores y NNA, y al II Estudio Nacional de la Discapacidad (ENDIS), ambos con codificación ISO-8859-1. Luego, se estandarizaron los nombres de las columnas a un formato uniforme para facilitar el manejo y análisis de la información. Posteriormente, ambos datasets fueron combinados usando únicamente las columnas comunes, creando así una base unificada y consistente.

Se realizó una imputación sencilla para tratar los datos faltantes, utilizando la mediana en columnas numéricas y el valor "Desconocido" en variables categóricas. Además, se añadió una nueva variable categórica denominada grupo_edad, segmentando las edades en grupos etarios claramente definidos.

Finalmente, se generaron dos DataFrames adicionales con resúmenes específicos:

a. Un conteo de personas según región y sexo.

b. Una tabla pivote que muestra la distribución regional según presencia o ausencia de discapacidad.



**2. Problemas detectados y cómo se abordaron**

1. **Problemas de delimitación de CSV**: Inicialmente tuve dificultades con la carga de los archivos, debido a una mala identificación del delimitador. Esto se solucionó identificando manualmente el separador.

2. **Valores faltantes y erróneos**: Existían numerosos valores faltantes, así como registros etiquetados como "#¡NULO!". Para abordar esta situación se decidió realizar imputaciones numéricas usando la mediana (para evitar efectos de valores extremos) y sustituir faltantes en variables categóricas por "Desconocido".

3. **Formato inconsistente en columnas**: La diversidad de formatos en nombres de columnas dificultaba análisis unificados. Por ello, se normalizó toda la nomenclatura usando formato snake_case.

**3. Preparación final del dataset y posibles usos posteriores.**

El dataset final resultante quedó limpio, consolidado y enriquecido, con una nueva variable de agrupación etaria. Este dataset, junto con los DataFrames resumen generados (por región, sexo y discapacidad), permite realizar análisis exploratorios rápidos y generar visualizaciones clave para reportes e informes.

Entre los usos posibles están:

-  Análisis descriptivos y exploratorios para identificar patrones y tendencias demográficas.

-  Modelos estadísticos o predictivos sobre factores asociados a la discapacidad o distribución demográfica regional.

-  Reportes de gestión pública o académicos que requieren información consolidada sobre población, discapacidad y características sociodemográficas.

